In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("enalis/tomatoes-dataset")




print("Path to dataset files:", path)

In [ ]:
print ("code will be running")

In [ ]:
import os

dataset_dir = os.path.join(path, "content","ieee-mbl-cls","train")
print("Kategori",os.listdir(dataset_dir))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # pakai preprocess_input ResNet50
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_gen.num_classes
print(f"Jumlah kelas: {num_classes}")


In [ ]:
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# outputs = Dense(num_classes, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=outputs)

# # Unfreeze 20–30 layer terakhir untuk fine-tuning
# for layer in base_model.layers[-30:]:
#     layer.trainable = True

# model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 tanpa top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Tambahkan layer kustom
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)  # bisa dikurangi biar ringan
x = Dropout(0.4)(x)
outputs = Dense(num_classes, activation='softmax')(x)

# Bangun model
model = Model(inputs=base_model.input, outputs=outputs)

# Fine-tune sebagian layer terakhir (10–20 layer)
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,        
    verbose=1
)


In [ ]:
val_loss, val_acc = model.evaluate(val_gen, verbose=1)
print(f"✅ Akurasi akhir: {val_acc:.4f}")


In [ ]:
# # Ambil jumlah kelas otomatis
# num_classes = train_gen.num_classes

# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.optimizers import Adam

# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# outputs = Dense(num_classes, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=outputs)

# # Unfreeze layer terakhir + fine-tuning
# for layer in base_model.layers[-30:]:
#     layer.trainable = True

# model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import numpy as np

def extract_features(model, generator):
    features = []
    labels = []
    for batch_x, batch_y in generator:
        if len(batch_x) == 0:
            continue
        feat = model.predict(batch_x, verbose=0)
        features.append(feat)
        labels.append(batch_y)
        if hasattr(generator, 'n') and len(features) * generator.batch_size >= generator.n:
            break
    if len(features) == 0:
        raise ValueError("Generator tidak menghasilkan batch apapun!")
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

X_train, y_train = extract_features(feature_model, train_gen)
X_val, y_val = extract_features(feature_model, val_gen)

# Flatten one-hot labels ke argmax
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

svm_model = SVC(kernel='rbf', C=10, gamma='scale')  # RBF kernel, hyperparameter bisa tuning

start = time.time()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_val)
end = time.time()

acc = accuracy_score(y_val, y_pred)
print(f"✅ Akurasi SVM RBF: {acc:.4f} | Waktu: {end-start:.2f} detik")
